In [ ]:
# Install external libraries
!pip install pandas
!pip install requests
!pip install python-whois
!pip install pyopenssl
!pip install beautifulsoup4
!pip install language-tool-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 2.5 MB/s eta 0:00:00


In [ ]:
import math
from collections import Counter
import re
import pandas as pd
from urllib.parse import urlparse, urljoin
import requests
import hashlib
import base64
import json
import whois
from datetime import datetime
import ssl
import socket
import OpenSSL
from bs4 import BeautifulSoup
import language_tool_python

# Replace with your Google API Key
API_KEY = 'API_KEY'
SAFE_BROWSING_URL = "https://safebrowsing.googleapis.com/v4/threatMatches:find"

# Function Definitions (Merged)

def calculate_entropy(url):
    frequency = Counter(url)
    total_characters = len(url)
    return -sum((count / total_characters) * math.log2(count / total_characters) for count in frequency.values())

def has_ip_address(url):
    ip_pattern = re.compile(r'\b(?:\d{1,3}\.){3}\d{1,3}\b')
    return bool(ip_pattern.search(url))

def count_dots_in_url(url):
    return url.count('.')

def count_hyphens_in_domain(url):
    domain = urlparse(url).netloc
    return domain.count('-')

def count_special_characters(url):
    special_characters = r'[@%$&!*^+=?/<>|~]'
    matches = re.findall(special_characters, url)
    return len(matches)

def check_url_safety(url):
    payload = {
        "client": {"clientId": "yourClientID", "clientVersion": "1.0"},
        "threatInfo": {
            "threatTypes": ["MALWARE", "SOCIAL_ENGINEERING"],
            "platformTypes": ["ANY_PLATFORM"],
            "threatEntryTypes": ["URL"],
            "threatEntries": [{"url": url}]
        }
    }
    try:
        response = requests.post(f'{SAFE_BROWSING_URL}?key={API_KEY}', json=payload)
        result = response.json()
        return "Suspicious tld: True" if 'matches' in result else "Suspicious tld: False"
    except requests.exceptions.RequestException as e:
        print(f"Error checking URL safety: {e}")
        return "Unknown"

def check_url_redirection(url):
    try:
        response = requests.get(url, allow_redirects=True)
        return len(response.history)
    except requests.exceptions.RequestException as e:
        print(f"Error checking URL redirection: {e}")
        return 0

def check_suspicious_ip(url):
    parsed_url = urlparse(url)
    ipv4_pattern = re.compile(r'^(?:\d{1,3}\.){3}\d{1,3}$')
    ipv6_pattern = re.compile(r'^[0-9a-fA-F:]{2,39}$')
    host = parsed_url.hostname
    if ipv4_pattern.match(host) or ipv6_pattern.match(host):
        return 1
    else:
        return 0

def check_https_token_in_url(url):
    parsed_url = urlparse(url)
    if 'https' in parsed_url.scheme or 'https' in parsed_url.path or 'https' in parsed_url.query:
        return 1
    else:
        return 0

def get_domain_age(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        creation_date = w.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        current_date = datetime.now()
        return (current_date - creation_date).days / 365
    except Exception as e:
        print(f"Error retrieving domain age: {e}")
        return None

def get_domain_expiration(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        expiration_date = w.expiration_date
        if isinstance(expiration_date, list):
            expiration_date = expiration_date[0]
        current_date = datetime.now()
        return (expiration_date - current_date).days
    except Exception as e:
        print(f"Error retrieving domain expiration: {e}")
        return None

def check_registrar(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        registrar = w.registrar
        suspicious_registrars = ["Namecheap Inc.", "GoDaddy.com, LLC", "PublicDomainRegistry.com"]
        if registrar and any(suspicious in registrar for suspicious in suspicious_registrars):
            return 1
        return 0
    except Exception as e:
        print(f"Error retrieving registrar: {e}")
        return None

def check_whois_privacy_protection(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        registrant_info = w.get('registrant', '') + ' ' + w.get('contact', '')
        privacy_protection_indicators = ["Privacy Protected", "Domains by Proxy"]
        if any(indicator in registrant_info for indicator in privacy_protection_indicators):
            return 1
        return 0
    except Exception as e:
        print(f"Error checking WHOIS privacy: {e}")
        return None

def check_owner_details(url):
    try:
        domain = url.split('/')[2]
        if domain.endswith('google.com'):
            return 0
        w = whois.whois(domain)
        if not w.get('registrant_name') or not w.get('registrant_email'):
            return 1
        return 0
    except Exception as e:
        print(f"Error checking owner details: {e}")
        return None

def check_ssl_certificate(url):
    try:
        domain = url.split('/')[2]
        conn = ssl.create_default_context().wrap_socket(socket.socket(), server_hostname=domain)
        conn.connect((domain, 443))
        cert = conn.getpeercert()
        not_after = datetime.strptime(cert['notAfter'], "%b %d %H:%M:%S %Y GMT")
        return not_after > datetime.now()
    except Exception as e:
        print(f"Error checking SSL certificate: {e}")
        return False

def get_certificate_validity(url):
    try:
        domain = url.split('/')[2]
        conn = ssl.create_default_context().wrap_socket(socket.socket(), server_hostname=domain)
        conn.connect((domain, 443))
        cert = conn.getpeercert()
        not_before = datetime.strptime(cert['notBefore'], "%b %d %H:%M:%S %Y GMT")
        not_after = datetime.strptime(cert['notAfter'], "%b %d %H:%M:%S %Y GMT")
        validity_duration = (not_after - not_before).days
        return validity_duration, not_after > datetime.now()
    except Exception as e:
        print(f"Error checking certificate validity: {e}")
        return None, False

def get_page_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup.get_text()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return ""

def check_grammar_and_spelling(text):
    tool = language_tool_python.LanguageTool('en-US')
    matches = tool.check(text)
    return len(matches), matches

def get_page_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a', href=True)
        base_domain = urlparse(url).netloc
        external_links = sum(1 for link in links if urlparse(urljoin(url, link['href'])).netloc != base_domain)
        internal_links = sum(1 for link in links if urlparse(urljoin(url, link['href'])).netloc == base_domain)
        return external_links, internal_links
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def detect_login_forms(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        forms = soup.find_all('form')
        return sum(1 for form in forms if form.find_all('input', {'name': lambda x: x and ('user' in x or 'password' in x)}))
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0

def detect_input_fields(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        input_fields = soup.find_all('input')
        input_types = {}
        for input_field in input_fields:
            input_type = input_field.get('type', 'text')
            input_types[input_type] = input_types.get(input_type, 0) + 1
        return len(input_fields), input_types
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, {}

def measure_javascript_length(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        script_tags = soup.find_all('script')
        return sum(len(script.string) for script in script_tags if script.string)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0

def detect_inline_scripts_and_iframes(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        inline_scripts_length = sum(len(script.string) for script in soup.find_all('script') if script.string)
        iframe_count = len(soup.find_all('iframe'))
        return inline_scripts_length, iframe_count
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def extract_title_and_meta_description_length(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        title_tag = soup.find('title')
        meta_description_tag = soup.find('meta', attrs={'name': 'description'})
        return len(title_tag.string) if title_tag else 0, len(meta_description_tag['content']) if meta_description_tag else 0
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def main():
    url = input("Enter a URL: ")

    total_features = 27
    extracted_features = 0

    # General URL analysis
    print("--- URL Analysis ---")
    try:
        print(f"URL Length: {len(url)}")
        print(f"Number of characters after '.com': {len(url.split('.com')[-1]) if '.com' in url else 0}")
        extracted_features += 1
        print(f"URL Entropy: {calculate_entropy(url):.2f}")
        extracted_features += 1
        print(f"Contains IP Address: {has_ip_address(url)}")
        extracted_features += 1
        print(f"Number of Dots: {count_dots_in_url(url)}")
        extracted_features += 1
        print(f"Number of Hyphens: {count_hyphens_in_domain(url)}")
        extracted_features += 1
        print(f"Special Characters: {count_special_characters(url)}")
        extracted_features += 1
        print(f"Google Safe Browsing: {check_url_safety(url)}")
        extracted_features += 1
        print(f"Number of redirects: {check_url_redirection(url)}")
        extracted_features += 1
        print(f"Suspicious IP in URL: {check_suspicious_ip(url)}")
        extracted_features += 1
        print(f"HTTPS token in URL: {check_https_token_in_url(url)}")
        extracted_features += 1
    except Exception as e:
        print(f"Error in URL Analysis: {e}")

    # WHOIS Analysis
    print("\n--- WHOIS Analysis ---")
    try:
        domain_age = get_domain_age(url)
        domain_expiration = get_domain_expiration(url)
        registrar_check = check_registrar(url)
        whois_privacy = check_whois_privacy_protection(url)
        owner_details = check_owner_details(url)

        print(f"Domain Age: {domain_age if domain_age else 'Unknown'} years")
        extracted_features += 1
        print(f"Domain Expiration Time: {domain_expiration if domain_expiration else 'Unknown'} days")
        extracted_features += 1
        print(f"Registrar check: {registrar_check}")
        extracted_features += 1
        print(f"WHOIS privacy protection: {whois_privacy}")
        extracted_features += 1
        print(f"Owner details check: {owner_details}")
        extracted_features += 1
    except Exception as e:
        print(f"Error in WHOIS Analysis: {e}")

    # SSL Certificate Analysis
    print("\n--- SSL Certificate Analysis ---")
    try:
        ssl_status = check_ssl_certificate(url)
        certificate_validity, certificate_active = get_certificate_validity(url)
        print(f"SSL Certificate Valid: {'Yes' if ssl_status else 'No'}")
        extracted_features += 1
        print(f"SSL Certificate Validity Period: {certificate_validity if certificate_validity else 'Unknown'} days")
        extracted_features += 1
    except Exception as e:
        print(f"Error in SSL Analysis: {e}")

    # Webpage Content and Analysis
    print("\n--- Webpage Analysis ---")
    try:
        content = get_page_content(url)
        if content:
            num_issues, issues = check_grammar_and_spelling(content)
            print(f"Number of grammatical or spelling issues: {num_issues}")
            extracted_features += 1
        else:
            print("Failed to retrieve page content.")

        external_links, internal_links = get_page_links(url)
        print(f"Number of external links: {external_links}")
        extracted_features += 1
        print(f"Number of internal links: {internal_links}")
        extracted_features += 1

        login_forms = detect_login_forms(url)
        input_fields, input_types = detect_input_fields(url)
        print(f"Number of login forms: {login_forms}")
        extracted_features += 1
        print(f"Number of input fields: {input_fields}")
        extracted_features += 1

        js_length = measure_javascript_length(url)
        inline_scripts_length, iframe_count = detect_inline_scripts_and_iframes(url)
        print(f"Total JavaScript length: {js_length} characters")
        extracted_features += 1
        print(f"Total length of inline JavaScript: {inline_scripts_length} characters")
        extracted_features += 1
        print(f"Number of iframe elements: {iframe_count}")
        extracted_features += 1

        title_length, meta_desc_length = extract_title_and_meta_description_length(url)
        print(f"Title Length: {title_length} characters")
        extracted_features += 1
        print(f"Meta Description Length: {meta_desc_length} characters")
        extracted_features += 1
    except Exception as e:
        print(f"Error in Webpage Analysis: {e}")

    print("\n--- Summary ---")
    print(f"Extracted Features: {extracted_features}/{total_features} successfully")

if __name__ == "__main__":
    main()


Enter a URL: https://colab.research.google.com/drive/1dK54kkN9BVhadPLuaQgQ15hhAJIvvomI?authuser=2#scrollTo=Qnx4XsDYHv2f
--- URL Analysis ---
URL Length: 106
Number of characters after '.com': 73
URL Entropy: 5.24
Contains IP Address: False
Number of Dots: 3
Number of Hyphens: 0
Special Characters: 7
Google Safe Browsing: Suspicious tld: False
Number of redirects: 0
Suspicious IP in URL: 0
HTTPS token in URL: 1

--- WHOIS Analysis ---
Domain Age: 27.28219178082192 years
Domain Expiration Time: 1363 days
Registrar check: 0
WHOIS privacy protection: 0
Owner details check: 0

--- SSL Certificate Analysis ---
SSL Certificate Valid: Yes
SSL Certificate Validity Period: 83 days

--- Webpage Analysis ---


INFO:language_tool_python.download_lt:Unzipping /tmp/tmpi5t7ljzf.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /root/.cache/language_tool_python.


Number of grammatical or spelling issues: 1
Number of external links: 1
Number of internal links: 0
Number of login forms: 0
Number of input fields: 0
Total JavaScript length: 64455 characters
Total length of inline JavaScript: 64453 characters
Number of iframe elements: 0
Title Length: 12 characters
Meta Description Length: 0 characters

--- Summary ---
Extracted Features: 27/27 successfully


In [ ]:
import math
from collections import Counter
import re
import requests
import hashlib
from urllib.parse import urlparse, urljoin
import whois
from datetime import datetime
import ssl
import socket
from bs4 import BeautifulSoup
import language_tool_python

# Replace with your Google API Key
API_KEY = 'API_KEY'
SAFE_BROWSING_URL = "https://safebrowsing.googleapis.com/v4/threatMatches:find"

def calculate_entropy(url):
    frequency = Counter(url)
    total_characters = len(url)
    return -sum((count / total_characters) * math.log2(count / total_characters) for count in frequency.values())

def has_ip_address(url):
    ip_pattern = re.compile(r'\b(?:\d{1,3}\.){3}\d{1,3}\b')
    return int(bool(ip_pattern.search(url)))

def count_dots_in_url(url):
    return url.count('.')

def count_hyphens_in_domain(url):
    domain = urlparse(url).netloc
    return domain.count('-')

def count_special_characters(url):
    special_characters = r'[@%$&!*^+=?/<>|~]'
    matches = re.findall(special_characters, url)
    return len(matches)

def check_url_safety(url):
    payload = {
        "client": {"clientId": "yourClientID", "clientVersion": "1.0"},
        "threatInfo": {
            "threatTypes": ["MALWARE", "SOCIAL_ENGINEERING"],
            "platformTypes": ["ANY_PLATFORM"],
            "threatEntryTypes": ["URL"],
            "threatEntries": [{"url": url}]
        }
    }
    try:
        response = requests.post(f'{SAFE_BROWSING_URL}?key={API_KEY}', json=payload)
        result = response.json()
        return int('matches' in result)
    except requests.exceptions.RequestException:
        return -1

def check_url_redirection(url):
    try:
        response = requests.get(url, allow_redirects=True)
        return len(response.history)
    except requests.exceptions.RequestException:
        return -1

def check_suspicious_ip(url):
    parsed_url = urlparse(url)
    ipv4_pattern = re.compile(r'^(?:\d{1,3}\.){3}\d{1,3}$')
    ipv6_pattern = re.compile(r'^[0-9a-fA-F:]{2,39}$')
    host = parsed_url.hostname
    return int(bool(ipv4_pattern.match(host) or ipv6_pattern.match(host)))

def check_https_token_in_url(url):
    return int('https' in urlparse(url).path or 'https' in urlparse(url).query)

def get_domain_age(url):
    try:
        domain = urlparse(url).netloc
        w = whois.whois(domain)
        creation_date = w.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        current_date = datetime.now()
        return (current_date - creation_date).days / 365
    except:
        return -1

def get_domain_expiration(url):
    try:
        domain = urlparse(url).netloc
        w = whois.whois(domain)
        expiration_date = w.expiration_date
        if isinstance(expiration_date, list):
            expiration_date = expiration_date[0]
        current_date = datetime.now()
        return (expiration_date - current_date).days
    except:
        return -1

def check_registrar(url):
    try:
        domain = urlparse(url).netloc
        w = whois.whois(domain)
        registrar = w.registrar
        suspicious_registrars = ["Namecheap Inc.", "GoDaddy.com, LLC", "PublicDomainRegistry.com"]
        return int(registrar and any(suspicious in registrar for suspicious in suspicious_registrars))
    except:
        return -1

def check_whois_privacy_protection(url):
    try:
        domain = urlparse(url).netloc
        w = whois.whois(domain)
        registrant_info = w.get('registrant', '') + ' ' + w.get('contact', '')
        privacy_protection_indicators = ["Privacy Protected", "Domains by Proxy"]
        return int(any(indicator in registrant_info for indicator in privacy_protection_indicators))
    except:
        return -1

def check_owner_details(url):
    try:
        domain = urlparse(url).netloc
        w = whois.whois(domain)
        return int(not w.get('registrant_name') or not w.get('registrant_email'))
    except:
        return -1

def check_ssl_certificate(url):
    try:
        domain = urlparse(url).netloc
        conn = ssl.create_default_context().wrap_socket(socket.socket(), server_hostname=domain)
        conn.connect((domain, 443))
        cert = conn.getpeercert()
        not_after = datetime.strptime(cert['notAfter'], "%b %d %H:%M:%S %Y GMT")
        return int(not_after > datetime.now())
    except:
        return 0

def get_certificate_validity(url):
    try:
        domain = urlparse(url).netloc
        conn = ssl.create_default_context().wrap_socket(socket.socket(), server_hostname=domain)
        conn.connect((domain, 443))
        cert = conn.getpeercert()
        not_before = datetime.strptime(cert['notBefore'], "%b %d %H:%M:%S %Y GMT")
        not_after = datetime.strptime(cert['notAfter'], "%b %d %H:%M:%S %Y GMT")
        return (not_after - not_before).days
    except:
        return -1

def get_page_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup.get_text()
    except:
        return ""

def check_grammar_and_spelling(text):
    try:
        tool = language_tool_python.LanguageTool('en-US')
        matches = tool.check(text)
        return len(matches)
    except:
        return -1

def get_page_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a', href=True)
        base_domain = urlparse(url).netloc
        external_links = sum(1 for link in links if urlparse(urljoin(url, link['href'])).netloc != base_domain)
        internal_links = sum(1 for link in links if urlparse(urljoin(url, link['href'])).netloc == base_domain)
        return external_links, internal_links
    except:
        return -1, -1

def detect_login_forms(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        forms = soup.find_all('form')
        return sum(1 for form in forms if form.find_all('input', {'name': lambda x: x and ('user' in x or 'password' in x)}))
    except:
        return -1

def detect_input_fields(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        input_fields = soup.find_all('input')
        return len(input_fields)
    except:
        return -1

def measure_javascript_length(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        script_tags = soup.find_all('script')
        return sum(len(script.string) for script in script_tags if script.string)
    except:
        return -1

def detect_inline_scripts_and_iframes(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        inline_scripts_length = sum(len(script.string) for script in soup.find_all('script') if script.string)
        iframe_count = len(soup.find_all('iframe'))
        return inline_scripts_length, iframe_count
    except:
        return -1, -1

def extract_title_and_meta_description_length(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        title_tag = soup.find('title')
        meta_description_tag = soup.find('meta', attrs={'name': 'description'})
        return len(title_tag.string) if title_tag else -1, len(meta_description_tag['content']) if meta_description_tag else -1
    except:
        return -1, -1

def main():
    url = input("Enter a URL: ")

    # Extract features into a 2D array
    features = [
        ["URL Length", len(url)],
        ["Characters After .com", len(url.split('.com')[-1]) if '.com' in url else 0],
        ["URL Entropy", calculate_entropy(url)],
        ["Contains IP Address", has_ip_address(url)],
        ["Number of Dots", count_dots_in_url(url)],
        ["Number of Hyphens", count_hyphens_in_domain(url)],
        ["Special Characters Count", count_special_characters(url)],
        ["Google Safe Browsing", check_url_safety(url)],
        ["Number of Redirects", check_url_redirection(url)],
        ["Suspicious IP in URL", check_suspicious_ip(url)],
        ["HTTPS Token in URL", check_https_token_in_url(url)],
        ["Domain Age (years)", get_domain_age(url)],
        ["Domain Expiration (days)", get_domain_expiration(url)],
        ["Suspicious Registrar", check_registrar(url)],
        ["WHOIS Privacy Protection", check_whois_privacy_protection(url)],
        ["Missing Owner Details", check_owner_details(url)],
        ["SSL Certificate Valid", check_ssl_certificate(url)],
        ["Certificate Validity Period (days)", get_certificate_validity(url)],
        ["Grammar and Spelling Issues", check_grammar_and_spelling(get_page_content(url))],
        ["Number of External Links", get_page_links(url)[0]],
        ["Number of Internal Links", get_page_links(url)[1]],
        ["Number of Login Forms", detect_login_forms(url)],
        ["Number of Input Fields", detect_input_fields(url)],
        ["JavaScript Length", measure_javascript_length(url)],
        ["Inline Scripts Length", detect_inline_scripts_and_iframes(url)[0]],
        ["Number of Iframes", detect_inline_scripts_and_iframes(url)[1]],
        ["Title Length", extract_title_and_meta_description_length(url)[0]],
        ["Meta Description Length", extract_title_and_meta_description_length(url)[1]],
        ["Page Content Length", len(get_page_content(url))]
    ]

    print("\nExtracted Features:")
    for feature in features:
        print(f"{feature[0]}: {feature[1]}")

if __name__ == "__main__":
    main()


Enter a URL: https://colab.research.google.com/drive/1dK54kkN9BVhadPLuaQgQ15hhAJIvvomI?authuser=2#scrollTo=x-NrMID1LhGQ

Extracted Features:
URL Length: 106
Characters After .com: 73
URL Entropy: 5.1729580736058525
Contains IP Address: 0
Number of Dots: 3
Number of Hyphens: 0
Special Characters Count: 7
Google Safe Browsing: 0
Number of Redirects: 0
Suspicious IP in URL: 0
HTTPS Token in URL: 0
Domain Age (years): 27.28219178082192
Domain Expiration (days): 1363
Suspicious Registrar: 0
WHOIS Privacy Protection: 0
Missing Owner Details: 1
SSL Certificate Valid: 1
Certificate Validity Period (days): 83
Grammar and Spelling Issues: 1
Number of External Links: 1
Number of Internal Links: 0
Number of Login Forms: 0
Number of Input Fields: 0
JavaScript Length: 64465
Inline Scripts Length: 64450
Number of Iframes: 0
Title Length: 12
Meta Description Length: -1
Page Content Length: 19


In [ ]:
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

# Load the trained RandomForest model and the scaler
with open('rf_best_model.pkl', 'rb') as model_file:
    rf_model = pickle.load(model_file)

with open('scaler.pkl', 'rb') as scaler_file:
    scaler = pickle.load(scaler_file)

# Predict function
def predict_from_features(features):
    # Get only required features based on the scaler's expected input size
    required_feature_count = scaler.mean_.shape[0]
    features = features[:required_feature_count]  # Truncate to required length

    # If fewer features are provided, pad with zeros
    if len(features) < required_feature_count:
        features += [0] * (required_feature_count - len(features))

    # Preprocess the features (e.g., scaling)
    features_scaled = scaler.transform([features])

    # Predict using the trained model
    prediction = rf_model.predict(features_scaled)
    prediction_proba = rf_model.predict_proba(features_scaled)

    # Return the prediction and probabilities
    return prediction[0], prediction_proba[0]

# Example usage
if __name__ == "__main__":
    # Example features array (replace with actual feature extraction logic)
    features = [
        106.0,  # URL length
        5.1729580736058525,   # URL entropy
        0.0,   # Contains IP address
        3.0,   # Count dots
        0.0,   # Contains hyphens
        7.0,   # Count special characters
        0.0,   # Suspicious TLD
        0.0,   # URL redirection
        1.0,   # Contains HTTPS token
        27.28219178082192, # Domain age
        1363.0, # Domain expiration
        1.0,   # Privacy protection
        1.0,  # SSL validity
        1.0,   # External links
        0.0,   # Internal links
        0.0,   # Login forms
        0.0,   # Input fields
        0.0,   # Hidden elements
        64465.0,# JavaScript length
        64450.0, # Inline script length
        0.0    # iFrames
    ]

    # Get the prediction
    label, probabilities = predict_from_features(features)

    # Output the result
    if label == 1:
        print("The URL is classified as: Phishy")
    else:
        print("The URL is classified as: Legitimate")

    print(f"Prediction Probabilities: Legitimate: {probabilities[0]:.2f}, Phishy: {probabilities[1]:.2f}")


The URL is classified as: Phishy
Prediction Probabilities: Legitimate: 0.41, Phishy: 0.59


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
import pandas as pd

a = pd.read_csv('updated_dataset2.csv')
a.head()

,url,url_length,url_entropy,contains_ip_address,count_dots,contains_hyphens,count_special_chars,suspicious_tld,url_redirection,contains_https_token,...,ssl_issuer,external_links,internal_links,login_forms,input_fields,hidden_elements,javascript_length,inline_script_length,iframes,status
0,http://www.crestonwood.com/router.php,37.0,3.787043,0.0,3.0,0.0,7.0,0.0,0.0,0.0,...,Let's Encrypt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,http://rgipt.ac.in,18.0,3.461320,0.0,2.0,0.0,5.0,0.0,0.0,0.0,...,Greater Manchester,62.0,5.0,0.0,6.0,0.0,3179.0,3097.0,0.0,0
2,http://www.mutuo.it,19.0,3.260828,0.0,2.0,0.0,5.0,0.0,0.0,0.0,...,Google Trust Services,76.0,3.0,0.0,12.0,0.0,3569.0,0.0,0.0,0
3,http://vamoaestudiarmedicina.blogspot.com/,42.0,4.195540,0.0,2.0,0.0,6.0,0.0,0.0,0.0,...,Google Trust Services,79.0,39.0,0.0,0.0,0.0,8085.0,8085.0,0.0,0
4,https://www.astrologyonline.eu/Astro_MemoNew/P...,56.0,4.344567,0.0,3.0,0.0,8.0,0.0,0.0,0.0,...,Let's Encrypt,4.0,9.0,0.0,0.0,0.0,1230.0,1230.0,0.0,0


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

# Load the dataset
file_path = 'updated_dataset2.csv'
dataset = pd.read_csv(file_path)

# Identify non-numeric columns
non_numeric_columns = dataset.select_dtypes(include=['object']).columns

# Convert non-numeric columns to numeric using Label Encoding
label_encoders = {}
for column in non_numeric_columns:
    if column != 'url':  # Exclude 'url' from encoding as it will not be used for modeling
        le = LabelEncoder()
        dataset[column] = le.fit_transform(dataset[column].astype(str))
        label_encoders[column] = le

# Handle missing values
imputer = SimpleImputer(strategy='mean')
dataset.iloc[:, 1:] = imputer.fit_transform(dataset.iloc[:, 1:])

# Separate features and target variable
X = dataset.drop(columns=['url', 'status'])
y = dataset['status']

# Apply SMOTE for class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_resampled)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_resampled, test_size=0.2, random_state=42)

# Hyperparameter tuning for Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf_grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='recall', n_jobs=-1)
rf_grid_search.fit(X_train, y_train)
rf_best_model = rf_grid_search.best_estimator_

# Train Logistic Regression
log_reg_classifier = LogisticRegression(random_state=42, max_iter=500)
log_reg_classifier.fit(X_train, y_train)

# Predictions
rf_predictions = rf_best_model.predict(X_test)
log_reg_predictions = log_reg_classifier.predict(X_test)

# Evaluation function
def evaluate_model(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=1)
    recall = recall_score(y_true, y_pred, zero_division=1)
    f1 = f1_score(y_true, y_pred, zero_division=1)

    print(f"{model_name} Performance:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}\n")

# Evaluate models
evaluate_model(y_test, rf_predictions, "Random Forest (Tuned)")
evaluate_model(y_test, log_reg_predictions, "Logistic Regression")


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


Random Forest (Tuned) Performance:
Accuracy: 0.94
Precision: 0.94
Recall: 0.95
F1 Score: 0.94

Logistic Regression Performance:
Accuracy: 0.82
Precision: 0.80
Recall: 0.87
F1 Score: 0.83



In [ ]:
import pickle

with open('rf_best_model.pkl', 'wb') as model_file:
    pickle.dump(rf_best_model, model_file)
with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)


In [ ]:
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

# Load the trained RandomForest model and the scaler
with open('rf_best_model.pkl', 'rb') as model_file:
    rf_model = pickle.load(model_file)

with open('scaler.pkl', 'rb') as scaler_file:
    scaler = pickle.load(scaler_file)

# Example feature extraction function (you should replace this with your real feature extraction logic)
def extract_features(url):
    """
    Placeholder for the URL feature extraction logic.
    Replace this with your actual feature extraction implementation.
    """
    # Example feature values based on the dataset
    return [
        len(url),  # URL length
        3.5,  # URL entropy
        0,  # Contains IP address
        url.count('.'),  # Count dots
        1 if '-' in url else 0,  # Contains hyphens
        sum(not char.isalnum() for char in url),  # Count special characters
        1 if any(tld in url for tld in ['.ru', '.cn']) else 0,  # Suspicious TLD
        1 if '//' in url.split('/')[2] else 0,  # URL redirection
        1 if 'https' in url.split(':')[0] else 0,  # Contains HTTPS token
        365,  # Domain age (default value if unknown)
        200,  # Domain expiration (default value if unknown)
        1,  # Privacy protection (example default)
        50,  # SSL validity (default value)
        0,  # External links (default value)
        5,  # Internal links (default value)
        0,  # Login forms (example default)
        0,  # Input fields (default value)
        0,  # Hidden elements (default value)
        1000,  # JavaScript length (default value)
        500,  # Inline script length (default value)
        0,  # iFrames (default value)
    ]

# Predict function
def predict_url(url):
    # Extract features
    features = extract_features(url)

    # Preprocess the features (e.g., scaling)
    features_scaled = scaler.transform([features])

    # Predict using the trained model
    prediction = rf_model.predict(features_scaled)
    prediction_proba = rf_model.predict_proba(features_scaled)

    # Return the prediction and probabilities
    return prediction[0], prediction_proba[0]

# Example usage
if __name__ == "__main__":
    # Input URL to check
    url_to_check = "http://example.com/login"

    # Get the prediction
    label, probabilities = predict_url(url_to_check)

    # Output the result
    if label == 1:
        print(f"The URL '{url_to_check}' is classified as: Phishy")
    else:
        print(f"The URL '{url_to_check}' is classified as: Legitimate")

    print(f"Prediction Probabilities: Legitimate: {probabilities[0]:.2f}, Phishy: {probabilities[1]:.2f}")




---
Again Complete feature Extraction Code:


In [ ]:
!pip install pandas requests whois-python pyopenssl beautifulsoup4 language-tool-python


ERROR: Could not find a version that satisfies the requirement whois-python (from versions: none)
ERROR: No matching distribution found for whois-python


In [ ]:
pip install whois-python

ERROR: Could not find a version that satisfies the requirement whois-python (from versions: none)
ERROR: No matching distribution found for whois-python


In [ ]:
import math
from collections import Counter
import re
import pandas as pd
from urllib.parse import urlparse, urljoin
import requests
import hashlib
import base64
import json
import whois
from datetime import datetime
import ssl
import socket
import OpenSSL
from bs4 import BeautifulSoup
import language_tool_python

# Replace with your Google API Key
API_KEY = 'API_KEY'
SAFE_BROWSING_URL = "https://safebrowsing.googleapis.com/v4/threatMatches:find"

# Function Definitions (Merged)

def calculate_entropy(url):
    frequency = Counter(url)
    total_characters = len(url)
    return -sum((count / total_characters) * math.log2(count / total_characters) for count in frequency.values())

def has_ip_address(url):
    ip_pattern = re.compile(r'\b(?:\d{1,3}\.){3}\d{1,3}\b')
    return bool(ip_pattern.search(url))

def count_dots_in_url(url):
    return url.count('.')

def count_hyphens_in_domain(url):
    domain = urlparse(url).netloc
    return domain.count('-')

def count_special_characters(url):
    special_characters = r'[@%$&!*^+=?/<>|~]'
    matches = re.findall(special_characters, url)
    return len(matches)

def check_url_safety(url):
    payload = {
        "client": {"clientId": "yourClientID", "clientVersion": "1.0"},
        "threatInfo": {
            "threatTypes": ["MALWARE", "SOCIAL_ENGINEERING"],
            "platformTypes": ["ANY_PLATFORM"],
            "threatEntryTypes": ["URL"],
            "threatEntries": [{"url": url}]
        }
    }
    try:
        response = requests.post(f'{SAFE_BROWSING_URL}?key={API_KEY}', json=payload)
        result = response.json()
        return "Suspicious tld: True" if 'matches' in result else "Suspicious tld: False"
    except requests.exceptions.RequestException as e:
        print(f"Error checking URL safety: {e}")
        return "Unknown"

def check_url_redirection(url):
    try:
        response = requests.get(url, allow_redirects=True)
        return len(response.history)
    except requests.exceptions.RequestException as e:
        print(f"Error checking URL redirection: {e}")
        return 0

def check_suspicious_ip(url):
    parsed_url = urlparse(url)
    ipv4_pattern = re.compile(r'^(?:\d{1,3}\.){3}\d{1,3}$')
    ipv6_pattern = re.compile(r'^[0-9a-fA-F:]{2,39}$')
    host = parsed_url.hostname
    if ipv4_pattern.match(host) or ipv6_pattern.match(host):
        return 1
    else:
        return 0

def check_https_token_in_url(url):
    parsed_url = urlparse(url)
    if 'https' in parsed_url.scheme or 'https' in parsed_url.path or 'https' in parsed_url.query:
        return 1
    else:
        return 0

def get_domain_age(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        creation_date = w.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        current_date = datetime.now()
        return (current_date - creation_date).days / 365
    except Exception as e:
        print(f"Error retrieving domain age: {e}")
        return None

def get_domain_expiration(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        expiration_date = w.expiration_date
        if isinstance(expiration_date, list):
            expiration_date = expiration_date[0]
        current_date = datetime.now()
        return (expiration_date - current_date).days
    except Exception as e:
        print(f"Error retrieving domain expiration: {e}")
        return None

def check_registrar(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        registrar = w.registrar
        suspicious_registrars = ["Namecheap Inc.", "GoDaddy.com, LLC", "PublicDomainRegistry.com"]
        if registrar and any(suspicious in registrar for suspicious in suspicious_registrars):
            return 1
        return 0
    except Exception as e:
        print(f"Error retrieving registrar: {e}")
        return None

def check_whois_privacy_protection(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        registrant_info = w.get('registrant', '') + ' ' + w.get('contact', '')
        privacy_protection_indicators = ["Privacy Protected", "Domains by Proxy"]
        if any(indicator in registrant_info for indicator in privacy_protection_indicators):
            return 1
        return 0
    except Exception as e:
        print(f"Error checking WHOIS privacy: {e}")
        return None

def check_owner_details(url):
    try:
        domain = url.split('/')[2]
        if domain.endswith('google.com'):
            return 0
        w = whois.whois(domain)
        if not w.get('registrant_name') or not w.get('registrant_email'):
            return 1
        return 0
    except Exception as e:
        print(f"Error checking owner details: {e}")
        return None

def check_ssl_certificate(url):
    try:
        domain = url.split('/')[2]
        conn = ssl.create_default_context().wrap_socket(socket.socket(), server_hostname=domain)
        conn.connect((domain, 443))
        cert = conn.getpeercert()
        not_after = datetime.strptime(cert['notAfter'], "%b %d %H:%M:%S %Y GMT")
        return not_after > datetime.now()
    except Exception as e:
        print(f"Error checking SSL certificate: {e}")
        return False

def get_certificate_validity(url):
    try:
        domain = url.split('/')[2]
        conn = ssl.create_default_context().wrap_socket(socket.socket(), server_hostname=domain)
        conn.connect((domain, 443))
        cert = conn.getpeercert()
        not_before = datetime.strptime(cert['notBefore'], "%b %d %H:%M:%S %Y GMT")
        not_after = datetime.strptime(cert['notAfter'], "%b %d %H:%M:%S %Y GMT")
        validity_duration = (not_after - not_before).days
        return validity_duration, not_after > datetime.now()
    except Exception as e:
        print(f"Error checking certificate validity: {e}")
        return None, False

def get_page_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup.get_text()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return ""

def check_grammar_and_spelling(text):
    tool = language_tool_python.LanguageTool('en-US')
    matches = tool.check(text)
    return len(matches), matches

def get_page_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a', href=True)
        base_domain = urlparse(url).netloc
        external_links = sum(1 for link in links if urlparse(urljoin(url, link['href'])).netloc != base_domain)
        internal_links = sum(1 for link in links if urlparse(urljoin(url, link['href'])).netloc == base_domain)
        return external_links, internal_links
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def detect_login_forms(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        forms = soup.find_all('form')
        return sum(1 for form in forms if form.find_all('input', {'name': lambda x: x and ('user' in x or 'password' in x)}))
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0

def detect_input_fields(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        input_fields = soup.find_all('input')
        input_types = {}
        for input_field in input_fields:
            input_type = input_field.get('type', 'text')
            input_types[input_type] = input_types.get(input_type, 0) + 1
        return len(input_fields), input_types
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, {}

def measure_javascript_length(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        script_tags = soup.find_all('script')
        return sum(len(script.string) for script in script_tags if script.string)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0

def detect_inline_scripts_and_iframes(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        inline_scripts_length = sum(len(script.string) for script in soup.find_all('script') if script.string)
        iframe_count = len(soup.find_all('iframe'))
        return inline_scripts_length, iframe_count
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def extract_title_and_meta_description_length(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        title_tag = soup.find('title')
        meta_description_tag = soup.find('meta', attrs={'name': 'description'})
        return len(title_tag.string) if title_tag else 0, len(meta_description_tag['content']) if meta_description_tag else 0
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def main():
    url = input("Enter a URL: ")

    total_features = 27
    extracted_features = 0

    # General URL analysis
    print("--- URL Analysis ---")
    try:
        print(f"URL Length: {len(url)}")
        print(f"Number of characters after '.com': {len(url.split('.com')[-1]) if '.com' in url else 0}")
        extracted_features += 1
        print(f"URL Entropy: {calculate_entropy(url):.2f}")
        extracted_features += 1
        print(f"Contains IP Address: {has_ip_address(url)}")
        extracted_features += 1
        print(f"Number of Dots: {count_dots_in_url(url)}")
        extracted_features += 1
        print(f"Number of Hyphens: {count_hyphens_in_domain(url)}")
        extracted_features += 1
        print(f"Special Characters: {count_special_characters(url)}")
        extracted_features += 1
        print(f"Google Safe Browsing: {check_url_safety(url)}")
        extracted_features += 1
        print(f"Number of redirects: {check_url_redirection(url)}")
        extracted_features += 1
        print(f"Suspicious IP in URL: {check_suspicious_ip(url)}")
        extracted_features += 1
        print(f"HTTPS token in URL: {check_https_token_in_url(url)}")
        extracted_features += 1
    except Exception as e:
        print(f"Error in URL Analysis: {e}")

    # WHOIS Analysis
    print("\n--- WHOIS Analysis ---")
    try:
        domain_age = get_domain_age(url)
        domain_expiration = get_domain_expiration(url)
        registrar_check = check_registrar(url)
        whois_privacy = check_whois_privacy_protection(url)
        owner_details = check_owner_details(url)

        print(f"Domain Age: {domain_age if domain_age else 'Unknown'} years")
        extracted_features += 1
        print(f"Domain Expiration Time: {domain_expiration if domain_expiration else 'Unknown'} days")
        extracted_features += 1
        print(f"Registrar check: {registrar_check}")
        extracted_features += 1
        print(f"WHOIS privacy protection: {whois_privacy}")
        extracted_features += 1
        print(f"Owner details check: {owner_details}")
        extracted_features += 1
    except Exception as e:
        print(f"Error in WHOIS Analysis: {e}")

    # SSL Certificate Analysis
    print("\n--- SSL Certificate Analysis ---")
    try:
        ssl_status = check_ssl_certificate(url)
        certificate_validity, certificate_active = get_certificate_validity(url)
        print(f"SSL Certificate Valid: {'Yes' if ssl_status else 'No'}")
        extracted_features += 1
        print(f"SSL Certificate Validity Period: {certificate_validity if certificate_validity else 'Unknown'} days")
        extracted_features += 1
    except Exception as e:
        print(f"Error in SSL Analysis: {e}")

    # Webpage Content and Analysis
    print("\n--- Webpage Analysis ---")
    try:
        content = get_page_content(url)
        if content:
            num_issues, issues = check_grammar_and_spelling(content)
            print(f"Number of grammatical or spelling issues: {num_issues}")
            extracted_features += 1
        else:
            print("Failed to retrieve page content.")

        external_links, internal_links = get_page_links(url)
        print(f"Number of external links: {external_links}")
        extracted_features += 1
        print(f"Number of internal links: {internal_links}")
        extracted_features += 1

        login_forms = detect_login_forms(url)
        input_fields, input_types = detect_input_fields(url)
        print(f"Number of login forms: {login_forms}")
        extracted_features += 1
        print(f"Number of input fields: {input_fields}")
        extracted_features += 1

        js_length = measure_javascript_length(url)
        inline_scripts_length, iframe_count = detect_inline_scripts_and_iframes(url)
        print(f"Total JavaScript length: {js_length} characters")
        extracted_features += 1
        print(f"Total length of inline JavaScript: {inline_scripts_length} characters")
        extracted_features += 1
        print(f"Number of iframe elements: {iframe_count}")
        extracted_features += 1

        title_length, meta_desc_length = extract_title_and_meta_description_length(url)
        print(f"Title Length: {title_length} characters")
        extracted_features += 1
        print(f"Meta Description Length: {meta_desc_length} characters")
        extracted_features += 1
    except Exception as e:
        print(f"Error in Webpage Analysis: {e}")

    print("\n--- Summary ---")
    print(f"Extracted Features: {extracted_features}/{total_features} successfully")

if __name__ == "__main__":
    main()


Enter a URL: https://www.snapchat.com/
--- URL Analysis ---
URL Length: 25
Number of characters after '.com': 1
URL Entropy: 3.59
Contains IP Address: False
Number of Dots: 2
Number of Hyphens: 0
Special Characters: 3
Google Safe Browsing: Suspicious tld: False
Number of redirects: 0
Suspicious IP in URL: 0
HTTPS token in URL: 1

--- WHOIS Analysis ---
Domain Age: 12.835616438356164 years
Domain Expiration Time: 428 days
Registrar check: 0
WHOIS privacy protection: 0
Owner details check: 1

--- SSL Certificate Analysis ---
SSL Certificate Valid: Yes
SSL Certificate Validity Period: 364 days

--- Webpage Analysis ---
Error fetching the page: 403 Client Error: Forbidden for url: https://www.snapchat.com/
Failed to retrieve page content.
Error fetching the page: 403 Client Error: Forbidden for url: https://www.snapchat.com/
Number of external links: 0
Number of internal links: 0
Error fetching the page: 403 Client Error: Forbidden for url: https://www.snapchat.com/
Error fetching the page

In [ ]:
!apt-get update
!apt-get install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -f install -y
!wget -O /tmp/chromedriver.zip https://chromedriver.storage.googleapis.com/$(curl -sS https://chromedriver.storage.googleapis.com/LATEST_RELEASE)/chromedriver_linux64.zip
!unzip /tmp/chromedriver.zip -d /usr/local/bin/


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,566 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,517 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Pac

In [ ]:
!pip install selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 24.5 MB/s eta 0:00:00


In [ ]:
pip install python-whois


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 3.9 MB/s eta 0:00:00


In [ ]:
pip install language-tool-python

In [ ]:
import math
from collections import Counter
import re
import pandas as pd
from urllib.parse import urlparse, urljoin
import requests
import hashlib
import base64
import json
import whois
from datetime import datetime
import ssl
import socket
import OpenSSL
from bs4 import BeautifulSoup
import language_tool_python
import requests
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


# Replace with your Google API Key
API_KEY = 'API_KEY'
SAFE_BROWSING_URL = "https://safebrowsing.googleapis.com/v4/threatMatches:find"

# Function Definitions (Merged)

def calculate_entropy(url):
    frequency = Counter(url)
    total_characters = len(url)
    return -sum((count / total_characters) * math.log2(count / total_characters) for count in frequency.values())

def has_ip_address(url):
    ip_pattern = re.compile(r'\b(?:\d{1,3}\.){3}\d{1,3}\b')
    return bool(ip_pattern.search(url))

def count_dots_in_url(url):
    return url.count('.')

def count_hyphens_in_domain(url):
    domain = urlparse(url).netloc
    return domain.count('-')

def count_special_characters(url):
    special_characters = r'[@%$&!*^+=?/<>|~]'
    matches = re.findall(special_characters, url)
    return len(matches)

def check_url_safety(url):
    payload = {
        "client": {"clientId": "yourClientID", "clientVersion": "1.0"},
        "threatInfo": {
            "threatTypes": ["MALWARE", "SOCIAL_ENGINEERING"],
            "platformTypes": ["ANY_PLATFORM"],
            "threatEntryTypes": ["URL"],
            "threatEntries": [{"url": url}]
        }
    }
    try:
        response = requests.post(f'{SAFE_BROWSING_URL}?key={API_KEY}', json=payload)
        result = response.json()
        return "Suspicious tld: True" if 'matches' in result else "Suspicious tld: False"
    except requests.exceptions.RequestException as e:
        print(f"Error checking URL safety: {e}")
        return "Unknown"

def check_url_redirection(url):
    try:
        response = requests.get(url, allow_redirects=True)
        return len(response.history)
    except requests.exceptions.RequestException as e:
        print(f"Error checking URL redirection: {e}")
        return 0

def check_suspicious_ip(url):
    parsed_url = urlparse(url)
    ipv4_pattern = re.compile(r'^(?:\d{1,3}\.){3}\d{1,3}$')
    ipv6_pattern = re.compile(r'^[0-9a-fA-F:]{2,39}$')
    host = parsed_url.hostname
    if ipv4_pattern.match(host) or ipv6_pattern.match(host):
        return 1
    else:
        return 0

def check_https_token_in_url(url):
    parsed_url = urlparse(url)
    if 'https' in parsed_url.scheme or 'https' in parsed_url.path or 'https' in parsed_url.query:
        return 1
    else:
        return 0

def get_domain_age(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        creation_date = w.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        current_date = datetime.now()
        return (current_date - creation_date).days / 365
    except Exception as e:
        print(f"Error retrieving domain age: {e}")
        return None

def get_domain_expiration(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        expiration_date = w.expiration_date
        if isinstance(expiration_date, list):
            expiration_date = expiration_date[0]
        current_date = datetime.now()
        return (expiration_date - current_date).days
    except Exception as e:
        print(f"Error retrieving domain expiration: {e}")
        return None

def check_registrar(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        registrar = w.registrar
        suspicious_registrars = ["Namecheap Inc.", "GoDaddy.com, LLC", "PublicDomainRegistry.com"]
        if registrar and any(suspicious in registrar for suspicious in suspicious_registrars):
            return 1
        return 0
    except Exception as e:
        print(f"Error retrieving registrar: {e}")
        return None

def check_whois_privacy_protection(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        registrant_info = w.get('registrant', '') + ' ' + w.get('contact', '')
        privacy_protection_indicators = ["Privacy Protected", "Domains by Proxy"]
        if any(indicator in registrant_info for indicator in privacy_protection_indicators):
            return 1
        return 0
    except Exception as e:
        print(f"Error checking WHOIS privacy: {e}")
        return None

def check_owner_details(url):
    try:
        domain = url.split('/')[2]
        if domain.endswith('google.com'):
            return 0
        w = whois.whois(domain)
        if not w.get('registrant_name') or not w.get('registrant_email'):
            return 1
        return 0
    except Exception as e:
        print(f"Error checking owner details: {e}")
        return None

def check_ssl_certificate(url):
    try:
        domain = url.split('/')[2]
        conn = ssl.create_default_context().wrap_socket(socket.socket(), server_hostname=domain)
        conn.connect((domain, 443))
        cert = conn.getpeercert()
        not_after = datetime.strptime(cert['notAfter'], "%b %d %H:%M:%S %Y GMT")
        return not_after > datetime.now()
    except Exception as e:
        print(f"Error checking SSL certificate: {e}")
        return False

def get_certificate_validity(url):
    try:
        domain = url.split('/')[2]
        conn = ssl.create_default_context().wrap_socket(socket.socket(), server_hostname=domain)
        conn.connect((domain, 443))
        cert = conn.getpeercert()
        not_before = datetime.strptime(cert['notBefore'], "%b %d %H:%M:%S %Y GMT")
        not_after = datetime.strptime(cert['notAfter'], "%b %d %H:%M:%S %Y GMT")
        validity_duration = (not_after - not_before).days
        return validity_duration, not_after > datetime.now()
    except Exception as e:
        print(f"Error checking certificate validity: {e}")
        return None, False

def get_page_content_with_selenium(url):
    # Configure headless Chrome
    # Configure Chrome options
  options = Options()
  options.add_argument("--headless")  # Run Chrome in headless mode
  options.add_argument("--no-sandbox")
  options.add_argument("--disable-dev-shm-usage")

  # Path to the ChromeDriver
  service = Service("/usr/local/bin/chromedriver")  # Update this path
  driver = webdriver.Chrome(service=service, options=options)
  try:
      driver.get(url)
      print("Page Title:", driver.title)  # Print the page title
      page_content = driver.page_source  # Get the page source
      print("Page content retrieved successfully!")
  except Exception as e:
      print("Error accessing page:", e)
  finally:
      driver.quit()

def check_grammar_and_spelling(text):
    tool = language_tool_python.LanguageTool('en-US')
    matches = tool.check(text)
    return len(matches), matches

def get_page_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a', href=True)
        base_domain = urlparse(url).netloc
        external_links = sum(1 for link in links if urlparse(urljoin(url, link['href'])).netloc != base_domain)
        internal_links = sum(1 for link in links if urlparse(urljoin(url, link['href'])).netloc == base_domain)
        return external_links, internal_links
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def detect_login_forms(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        forms = soup.find_all('form')
        return sum(1 for form in forms if form.find_all('input', {'name': lambda x: x and ('user' in x or 'password' in x)}))
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0

def detect_input_fields(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        input_fields = soup.find_all('input')
        input_types = {}
        for input_field in input_fields:
            input_type = input_field.get('type', 'text')
            input_types[input_type] = input_types.get(input_type, 0) + 1
        return len(input_fields), input_types
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, {}

def measure_javascript_length(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        script_tags = soup.find_all('script')
        return sum(len(script.string) for script in script_tags if script.string)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0

def detect_inline_scripts_and_iframes(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        inline_scripts_length = sum(len(script.string) for script in soup.find_all('script') if script.string)
        iframe_count = len(soup.find_all('iframe'))
        return inline_scripts_length, iframe_count
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def extract_title_and_meta_description_length(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        title_tag = soup.find('title')
        meta_description_tag = soup.find('meta', attrs={'name': 'description'})
        return len(title_tag.string) if title_tag else 0, len(meta_description_tag['content']) if meta_description_tag else 0
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def main():
    url = input("Enter a URL: ")

    total_features = 27
    extracted_features = 0

    # General URL analysis
    print("--- URL Analysis ---")
    try:
        print(f"URL Length: {len(url)}")
        print(f"Number of characters after '.com': {len(url.split('.com')[-1]) if '.com' in url else 0}")
        extracted_features += 1
        print(f"URL Entropy: {calculate_entropy(url):.2f}")
        extracted_features += 1
        print(f"Contains IP Address: {has_ip_address(url)}")
        extracted_features += 1
        print(f"Number of Dots: {count_dots_in_url(url)}")
        extracted_features += 1
        print(f"Number of Hyphens: {count_hyphens_in_domain(url)}")
        extracted_features += 1
        print(f"Special Characters: {count_special_characters(url)}")
        extracted_features += 1
        print(f"Google Safe Browsing: {check_url_safety(url)}")
        extracted_features += 1
        print(f"Number of redirects: {check_url_redirection(url)}")
        extracted_features += 1
        print(f"Suspicious IP in URL: {check_suspicious_ip(url)}")
        extracted_features += 1
        print(f"HTTPS token in URL: {check_https_token_in_url(url)}")
        extracted_features += 1
    except Exception as e:
        print(f"Error in URL Analysis: {e}")

    # WHOIS Analysis
    print("\n--- WHOIS Analysis ---")
    try:
        domain_age = get_domain_age(url)
        domain_expiration = get_domain_expiration(url)
        registrar_check = check_registrar(url)
        whois_privacy = check_whois_privacy_protection(url)
        owner_details = check_owner_details(url)

        print(f"Domain Age: {domain_age if domain_age else 'Unknown'} years")
        extracted_features += 1
        print(f"Domain Expiration Time: {domain_expiration if domain_expiration else 'Unknown'} days")
        extracted_features += 1
        print(f"Registrar check: {registrar_check}")
        extracted_features += 1
        print(f"WHOIS privacy protection: {whois_privacy}")
        extracted_features += 1
        print(f"Owner details check: {owner_details}")
        extracted_features += 1
    except Exception as e:
        print(f"Error in WHOIS Analysis: {e}")

    # SSL Certificate Analysis
    print("\n--- SSL Certificate Analysis ---")
    try:
        ssl_status = check_ssl_certificate(url)
        certificate_validity, certificate_active = get_certificate_validity(url)
        print(f"SSL Certificate Valid: {'Yes' if ssl_status else 'No'}")
        extracted_features += 1
        print(f"SSL Certificate Validity Period: {certificate_validity if certificate_validity else 'Unknown'} days")
        extracted_features += 1
    except Exception as e:
        print(f"Error in SSL Analysis: {e}")

    # Webpage Content and Analysis
    print("\n--- Webpage Analysis ---")
    try:
        content = get_page_content(url)
        if content:
            num_issues, issues = check_grammar_and_spelling(content)
            print(f"Number of grammatical or spelling issues: {num_issues}")
            extracted_features += 1
        else:
            print("Failed to retrieve page content.")

        external_links, internal_links = get_page_links(url)
        print(f"Number of external links: {external_links}")
        extracted_features += 1
        print(f"Number of internal links: {internal_links}")
        extracted_features += 1

        login_forms = detect_login_forms(url)
        input_fields, input_types = detect_input_fields(url)
        print(f"Number of login forms: {login_forms}")
        extracted_features += 1
        print(f"Number of input fields: {input_fields}")
        extracted_features += 1

        js_length = measure_javascript_length(url)
        inline_scripts_length, iframe_count = detect_inline_scripts_and_iframes(url)
        print(f"Total JavaScript length: {js_length} characters")
        extracted_features += 1
        print(f"Total length of inline JavaScript: {inline_scripts_length} characters")
        extracted_features += 1
        print(f"Number of iframe elements: {iframe_count}")
        extracted_features += 1

        title_length, meta_desc_length = extract_title_and_meta_description_length(url)
        print(f"Title Length: {title_length} characters")
        extracted_features += 1
        print(f"Meta Description Length: {meta_desc_length} characters")
        extracted_features += 1
    except Exception as e:
        print(f"Error in Webpage Analysis: {e}")

    print("\n--- Summary ---")
    print(f"Extracted Features: {extracted_features}/{total_features} successfully")

if __name__ == "__main__":
    main()


Enter a URL: https://www.snapchat.com/
--- URL Analysis ---
URL Length: 25
Number of characters after '.com': 1
URL Entropy: 3.59
Contains IP Address: False
Number of Dots: 2
Number of Hyphens: 0
Special Characters: 3
Google Safe Browsing: Suspicious tld: False
Number of redirects: 0
Suspicious IP in URL: 0
HTTPS token in URL: 1

--- WHOIS Analysis ---
Domain Age: 12.835616438356164 years
Domain Expiration Time: 428 days
Registrar check: 0
WHOIS privacy protection: 0
Owner details check: 1

--- SSL Certificate Analysis ---
SSL Certificate Valid: Yes
SSL Certificate Validity Period: 364 days

--- Webpage Analysis ---
Error in Webpage Analysis: name 'get_page_content' is not defined

--- Summary ---
Extracted Features: 17/27 successfully


In [ ]:
!pip install pandas
!pip install requests
!pip install whois
!pip install python-whois
!pip install language-tool-python
!pip install beautifulsoup4
!pip install pyOpenSSL


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 2.0 MB/s eta 0:00:00


In [ ]:
import math
from collections import Counter
import re
import pandas as pd
from urllib.parse import urlparse, urljoin
import requests
import hashlib
import base64
import json
import whois
from datetime import datetime
import ssl
import socket
import OpenSSL
from bs4 import BeautifulSoup
import language_tool_python

# Replace with your Google API Key
API_KEY = 'API_KEY'
SAFE_BROWSING_URL = "https://safebrowsing.googleapis.com/v4/threatMatches:find"

# Function Definitions (Merged)

def count_characters_after_tld(url, tld=".com"):
    try:
        if tld in url:
            index = url.find(tld) + len(tld)  # Find the position right after the TLD
            return len(url) - index  # Calculate characters after the TLD
        return 0
    except Exception as e:
        print(f"Error counting characters after TLD: {e}")
        return 0

def calculate_entropy(url):
    frequency = Counter(url)
    total_characters = len(url)
    return -sum((count / total_characters) * math.log2(count / total_characters) for count in frequency.values())

def has_ip_address(url):
    ip_pattern = re.compile(r'\b(?:\d{1,3}\.){3}\d{1,3}\b')
    return bool(ip_pattern.search(url))

def count_dots_in_url(url):
    return url.count('.')

def count_hyphens_in_domain(url):
    domain = urlparse(url).netloc
    return domain.count('-')

def count_special_characters(url):
    special_characters = r'[@%$&!*^+=?/<>|~]'
    matches = re.findall(special_characters, url)
    return len(matches)

def check_url_safety(url):
    payload = {
        "client": {"clientId": "yourClientID", "clientVersion": "1.0"},
        "threatInfo": {
            "threatTypes": ["MALWARE", "SOCIAL_ENGINEERING"],
            "platformTypes": ["ANY_PLATFORM"],
            "threatEntryTypes": ["URL"],
            "threatEntries": [{"url": url}]
        }
    }
    try:
        response = requests.post(f'{SAFE_BROWSING_URL}?key={API_KEY}', json=payload)
        result = response.json()
        return "Suspicious tld: True" if 'matches' in result else "Suspicious tld: False"
    except requests.exceptions.RequestException as e:
        print(f"Error checking URL safety: {e}")
        return "Unknown"

def check_url_redirection(url):
    try:
        response = requests.get(url, allow_redirects=True)
        return len(response.history)
    except requests.exceptions.RequestException as e:
        print(f"Error checking URL redirection: {e}")
        return 0

def check_suspicious_ip(url):
    parsed_url = urlparse(url)
    ipv4_pattern = re.compile(r'^(?:\d{1,3}\.){3}\d{1,3}$')
    ipv6_pattern = re.compile(r'^[0-9a-fA-F:]{2,39}$')
    host = parsed_url.hostname
    if ipv4_pattern.match(host) or ipv6_pattern.match(host):
        return 1
    else:
        return 0

def check_https_token_in_url(url):
    parsed_url = urlparse(url)
    if 'https' in parsed_url.scheme or 'https' in parsed_url.path or 'https' in parsed_url.query:
        return 1
    else:
        return 0

def get_domain_age(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        creation_date = w.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        current_date = datetime.now()
        return (current_date - creation_date).days / 365
    except Exception as e:
        print(f"Error retrieving domain age: {e}")
        return None

def get_domain_expiration(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        expiration_date = w.expiration_date
        if isinstance(expiration_date, list):
            expiration_date = expiration_date[0]
        current_date = datetime.now()
        return (expiration_date - current_date).days
    except Exception as e:
        print(f"Error retrieving domain expiration: {e}")
        return None

def check_registrar(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        registrar = w.registrar
        suspicious_registrars = ["Namecheap Inc.", "GoDaddy.com, LLC", "PublicDomainRegistry.com"]
        if registrar and any(suspicious in registrar for suspicious in suspicious_registrars):
            return 1
        return 0
    except Exception as e:
        print(f"Error retrieving registrar: {e}")
        return None

def check_whois_privacy_protection(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        registrant_info = w.get('registrant', '') + ' ' + w.get('contact', '')
        privacy_protection_indicators = ["Privacy Protected", "Domains by Proxy"]
        if any(indicator in registrant_info for indicator in privacy_protection_indicators):
            return 1
        return 0
    except Exception as e:
        print(f"Error checking WHOIS privacy: {e}")
        return None

def check_owner_details(url):
    try:
        domain = url.split('/')[2]
        if domain.endswith('google.com'):
            return 0
        w = whois.whois(domain)
        if not w.get('registrant_name') or not w.get('registrant_email'):
            return 1
        return 0
    except Exception as e:
        print(f"Error checking owner details: {e}")
        return None

def check_ssl_certificate(url):
    try:
        domain = url.split('/')[2]
        conn = ssl.create_default_context().wrap_socket(socket.socket(), server_hostname=domain)
        conn.connect((domain, 443))
        cert = conn.getpeercert()
        not_after = datetime.strptime(cert['notAfter'], "%b %d %H:%M:%S %Y GMT")
        return not_after > datetime.now()
    except Exception as e:
        print(f"Error checking SSL certificate: {e}")
        return False

def get_certificate_validity(url):
    try:
        domain = url.split('/')[2]
        conn = ssl.create_default_context().wrap_socket(socket.socket(), server_hostname=domain)
        conn.connect((domain, 443))
        cert = conn.getpeercert()
        not_before = datetime.strptime(cert['notBefore'], "%b %d %H:%M:%S %Y GMT")
        not_after = datetime.strptime(cert['notAfter'], "%b %d %H:%M:%S %Y GMT")
        validity_duration = (not_after - not_before).days
        return validity_duration, not_after > datetime.now()
    except Exception as e:
        print(f"Error checking certificate validity: {e}")
        return None, False

def get_page_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup.get_text()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return ""

def check_grammar_and_spelling(text):
    tool = language_tool_python.LanguageTool('en-US')
    matches = tool.check(text)
    return len(matches), matches

def get_page_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a', href=True)
        base_domain = urlparse(url).netloc
        external_links = sum(1 for link in links if urlparse(urljoin(url, link['href'])).netloc != base_domain)
        internal_links = sum(1 for link in links if urlparse(urljoin(url, link['href'])).netloc == base_domain)
        return external_links, internal_links
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def detect_login_forms(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        forms = soup.find_all('form')
        return sum(1 for form in forms if form.find_all('input', {'name': lambda x: x and ('user' in x or 'password' in x)}))
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0

def detect_input_fields(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        input_fields = soup.find_all('input')
        input_types = {}
        for input_field in input_fields:
            input_type = input_field.get('type', 'text')
            input_types[input_type] = input_types.get(input_type, 0) + 1
        return len(input_fields), input_types
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, {}

def measure_javascript_length(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        script_tags = soup.find_all('script')
        return sum(len(script.string) for script in script_tags if script.string)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0

def detect_inline_scripts_and_iframes(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        inline_scripts_length = sum(len(script.string) for script in soup.find_all('script') if script.string)
        iframe_count = len(soup.find_all('iframe'))
        return inline_scripts_length, iframe_count
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def extract_title_and_meta_description_length(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        title_tag = soup.find('title')
        meta_description_tag = soup.find('meta', attrs={'name': 'description'})
        return len(title_tag.string) if title_tag else 0, len(meta_description_tag['content']) if meta_description_tag else 0
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def main():
    url = input("Enter a URL: ")

    total_features = 27
    extracted_features = 0

    # General URL analysis
    print("--- URL Analysis ---")
    try:
        print(f"URL Length: {len(url)}")
        print(f"Number of characters after '.com': {count_characters_after_tld(url)}")
        extracted_features += 1
        print(f"URL Entropy: {calculate_entropy(url):.2f}")
        extracted_features += 1
        print(f"Contains IP Address: {has_ip_address(url)}")
        extracted_features += 1
        print(f"Number of Dots: {count_dots_in_url(url)}")
        extracted_features += 1
        print(f"Number of Hyphens: {count_hyphens_in_domain(url)}")
        extracted_features += 1
        print(f"Special Characters: {count_special_characters(url)}")
        extracted_features += 1
        print(f"Google Safe Browsing: {check_url_safety(url)}")
        extracted_features += 1
        print(f"Number of redirects: {check_url_redirection(url)}")
        extracted_features += 1
        print(f"Suspicious IP in URL: {check_suspicious_ip(url)}")
        extracted_features += 1
        print(f"HTTPS token in URL: {check_https_token_in_url(url)}")
        extracted_features += 1
    except Exception as e:
        print(f"Error in URL Analysis: {e}")

    # WHOIS Analysis
    print("\n--- WHOIS Analysis ---")
    try:
        domain_age = get_domain_age(url)
        if domain_age is not None:
            print(f"Domain Age: {domain_age:.2f} years")
            extracted_features += 1

        domain_expiration = get_domain_expiration(url)
        if domain_expiration is not None:
            print(f"Domain Expiration Time: {domain_expiration} days")
            extracted_features += 1

        registrar_check = check_registrar(url)
        if registrar_check is not None:
            print(f"Registrar check: {registrar_check}")
            extracted_features += 1

        whois_privacy = check_whois_privacy_protection(url)
        if whois_privacy is not None:
            print(f"WHOIS privacy protection: {whois_privacy}")
            extracted_features += 1

        owner_details = check_owner_details(url)
        if owner_details is not None:
            print(f"Owner details check: {owner_details}")
            extracted_features += 1
    except Exception as e:
        print(f"Error in WHOIS Analysis: {e}")

    # SSL Certificate Analysis
    print("\n--- SSL Certificate Analysis ---")
    try:
        ssl_status = check_ssl_certificate(url)
        if ssl_status:
            print(f"SSL Certificate Valid: {'Yes' if ssl_status else 'No'}")
            extracted_features += 1

        certificate_validity, certificate_active = get_certificate_validity(url)
        if certificate_validity:
            print(f"SSL Certificate Validity Period: {certificate_validity} days")
            extracted_features += 1
    except Exception as e:
        print(f"Error in SSL Analysis: {e}")

    # Webpage Content and Analysis
    print("\n--- Webpage Analysis ---")
    try:
        content = get_page_content(url)
        if content:
            num_issues, issues = check_grammar_and_spelling(content)
            print(f"Number of grammatical or spelling issues: {num_issues}")
            extracted_features += 1

            external_links, internal_links = get_page_links(url)
            print(f"Number of external links: {external_links}")
            extracted_features += 1
            print(f"Number of internal links: {internal_links}")
            extracted_features += 1

            login_forms = detect_login_forms(url)
            print(f"Number of login forms: {login_forms}")
            extracted_features += 1

            input_fields, input_types = detect_input_fields(url)
            print(f"Number of input fields: {input_fields}")
            extracted_features += 1

            js_length = measure_javascript_length(url)
            print(f"Total JavaScript length: {js_length} characters")
            extracted_features += 1

            inline_scripts_length, iframe_count = detect_inline_scripts_and_iframes(url)
            print(f"Total length of inline JavaScript: {inline_scripts_length} characters")
            extracted_features += 1
            print(f"Number of iframe elements: {iframe_count}")
            extracted_features += 1

            title_length, meta_desc_length = extract_title_and_meta_description_length(url)
            print(f"Title Length: {title_length} characters")
            extracted_features += 1
            print(f"Meta Description Length: {meta_desc_length} characters")
            extracted_features += 1
        else:
            print("Failed to retrieve page content.")
    except Exception as e:
        print(f"Error in Webpage Analysis: {e}")

    print("\n--- Summary ---")
    print(f"Extracted Features: {extracted_features}/{total_features} successfully")

if __name__ == "__main__":
    main()


Enter a URL: https://www.facebook.com/login/?next=https%3A%2F%2Fwww.facebook.com%2F
--- URL Analysis ---
URL Length: 70
Number of characters after '.com': 46
URL Entropy: 4.54
Contains IP Address: False
Number of Dots: 4
Number of Hyphens: 0
Special Characters: 10
Google Safe Browsing: Suspicious tld: False
Number of redirects: 0
Suspicious IP in URL: 0
HTTPS token in URL: 1

--- WHOIS Analysis ---
Domain Age: 27.77 years
Domain Expiration Time: 3013 days
Registrar check: 0
WHOIS privacy protection: 0
Owner details check: 1

--- SSL Certificate Analysis ---
SSL Certificate Valid: Yes
SSL Certificate Validity Period: 90 days

--- Webpage Analysis ---
Number of grammatical or spelling issues: 15
Number of external links: 18
Number of internal links: 29
Number of login forms: 1
Number of input fields: 23
Total JavaScript length: 48735 characters
Total length of inline JavaScript: 48734 characters
Number of iframe elements: 0
Title Length: 11 characters
Meta Description Length: 31 characte

In [ ]:
import math
from collections import Counter
import re
import pandas as pd
from urllib.parse import urlparse, urljoin
import requests
import hashlib
import base64
import json
import whois
from datetime import datetime
import ssl
import socket
import OpenSSL
from bs4 import BeautifulSoup
import language_tool_python

# Replace with your Google API Key
API_KEY = ''
SAFE_BROWSING_URL = "https://safebrowsing.googleapis.com/v4/threatMatches:find"

# Function Definitions (Merged)

def count_characters_after_tld(url, tld=".com"):
    try:
        if tld in url:
            index = url.find(tld) + len(tld)  # Find the position right after the TLD
            return len(url) - index  # Calculate characters after the TLD
        return 0
    except Exception as e:
        print(f"Error counting characters after TLD: {e}")
        return 0

def calculate_entropy(url):
    frequency = Counter(url)
    total_characters = len(url)
    return -sum((count / total_characters) * math.log2(count / total_characters) for count in frequency.values())

def has_ip_address(url):
    ip_pattern = re.compile(r'\b(?:\d{1,3}\.){3}\d{1,3}\b')
    return bool(ip_pattern.search(url))

def count_dots_in_url(url):
    return url.count('.')

def count_hyphens_in_domain(url):
    domain = urlparse(url).netloc
    return domain.count('-')

def count_special_characters(url):
    special_characters = r'[@%$&!*^+=?/<>|~]'
    matches = re.findall(special_characters, url)
    return len(matches)

def check_url_safety(url):
    payload = {
        "client": {"clientId": "yourClientID", "clientVersion": "1.0"},
        "threatInfo": {
            "threatTypes": ["MALWARE", "SOCIAL_ENGINEERING"],
            "platformTypes": ["ANY_PLATFORM"],
            "threatEntryTypes": ["URL"],
            "threatEntries": [{"url": url}]
        }
    }
    try:
        response = requests.post(f'{SAFE_BROWSING_URL}?key={API_KEY}', json=payload)
        result = response.json()
        return "Suspicious tld: True" if 'matches' in result else "Suspicious tld: False"
    except requests.exceptions.RequestException as e:
        print(f"Error checking URL safety: {e}")
        return "Unknown"

def check_url_redirection(url):
    try:
        response = requests.get(url, allow_redirects=True)
        return len(response.history)
    except requests.exceptions.RequestException as e:
        print(f"Error checking URL redirection: {e}")
        return 0

def check_suspicious_ip(url):
    parsed_url = urlparse(url)
    ipv4_pattern = re.compile(r'^(?:\d{1,3}\.){3}\d{1,3}$')
    ipv6_pattern = re.compile(r'^[0-9a-fA-F:]{2,39}$')
    host = parsed_url.hostname
    if ipv4_pattern.match(host) or ipv6_pattern.match(host):
        return 1
    else:
        return 0

def check_https_token_in_url(url):
    parsed_url = urlparse(url)
    if 'https' in parsed_url.scheme or 'https' in parsed_url.path or 'https' in parsed_url.query:
        return 1
    else:
        return 0

def get_domain_age(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        creation_date = w.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        current_date = datetime.now()
        return (current_date - creation_date).days / 365
    except Exception as e:
        print(f"Error retrieving domain age: {e}")
        return None

def get_domain_expiration(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        expiration_date = w.expiration_date
        if isinstance(expiration_date, list):
            expiration_date = expiration_date[0]
        current_date = datetime.now()
        return (expiration_date - current_date).days
    except Exception as e:
        print(f"Error retrieving domain expiration: {e}")
        return None

def check_registrar(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        registrar = w.registrar
        suspicious_registrars = ["Namecheap Inc.", "GoDaddy.com, LLC", "PublicDomainRegistry.com"]
        if registrar and any(suspicious in registrar for suspicious in suspicious_registrars):
            return 1
        return 0
    except Exception as e:
        print(f"Error retrieving registrar: {e}")
        return None

def check_whois_privacy_protection(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        registrant_info = w.get('registrant', '') + ' ' + w.get('contact', '')
        privacy_protection_indicators = ["Privacy Protected", "Domains by Proxy"]
        if any(indicator in registrant_info for indicator in privacy_protection_indicators):
            return 1
        return 0
    except Exception as e:
        print(f"Error checking WHOIS privacy: {e}")
        return None

def check_owner_details(url):
    try:
        domain = url.split('/')[2]
        if domain.endswith('google.com'):
            return 0
        w = whois.whois(domain)
        if not w.get('registrant_name') or not w.get('registrant_email'):
            return 1
        return 0
    except Exception as e:
        print(f"Error checking owner details: {e}")
        return None

def check_ssl_certificate(url):
    try:
        domain = url.split('/')[2]
        conn = ssl.create_default_context().wrap_socket(socket.socket(), server_hostname=domain)
        conn.connect((domain, 443))
        cert = conn.getpeercert()
        not_after = datetime.strptime(cert['notAfter'], "%b %d %H:%M:%S %Y GMT")
        return not_after > datetime.now()
    except Exception as e:
        print(f"Error checking SSL certificate: {e}")
        return False

def get_certificate_validity(url):
    try:
        domain = url.split('/')[2]
        conn = ssl.create_default_context().wrap_socket(socket.socket(), server_hostname=domain)
        conn.connect((domain, 443))
        cert = conn.getpeercert()
        not_before = datetime.strptime(cert['notBefore'], "%b %d %H:%M:%S %Y GMT")
        not_after = datetime.strptime(cert['notAfter'], "%b %d %H:%M:%S %Y GMT")
        validity_duration = (not_after - not_before).days
        return validity_duration, not_after > datetime.now()
    except Exception as e:
        print(f"Error checking certificate validity: {e}")
        return None, False

def get_page_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup.get_text()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return ""

def check_grammar_and_spelling(text):
    tool = language_tool_python.LanguageTool('en-US')
    matches = tool.check(text)
    return len(matches), matches

def get_page_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a', href=True)
        base_domain = urlparse(url).netloc
        external_links = sum(1 for link in links if urlparse(urljoin(url, link['href'])).netloc != base_domain)
        internal_links = sum(1 for link in links if urlparse(urljoin(url, link['href'])).netloc == base_domain)
        return external_links, internal_links
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def detect_login_forms(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        forms = soup.find_all('form')
        return sum(1 for form in forms if form.find_all('input', {'name': lambda x: x and ('user' in x or 'password' in x)}))
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0

def detect_input_fields(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        input_fields = soup.find_all('input')
        input_types = {}
        for input_field in input_fields:
            input_type = input_field.get('type', 'text')
            input_types[input_type] = input_types.get(input_type, 0) + 1
        return len(input_fields), input_types
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, {}

def measure_javascript_length(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        script_tags = soup.find_all('script')
        return sum(len(script.string) for script in script_tags if script.string)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0

def detect_inline_scripts_and_iframes(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        inline_scripts_length = sum(len(script.string) for script in soup.find_all('script') if script.string)
        iframe_count = len(soup.find_all('iframe'))
        return inline_scripts_length, iframe_count
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def extract_title_and_meta_description_length(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        title_tag = soup.find('title')
        meta_description_tag = soup.find('meta', attrs={'name': 'description'})
        return len(title_tag.string) if title_tag else 0, len(meta_description_tag['content']) if meta_description_tag else 0
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def main():
    url = input("Enter a URL: ")

    total_features = 27
    extracted_features = 0

    # General URL analysis
    print("--- URL Analysis ---")
    try:
        print(f"URL Length: {len(url)}")
        print(f"Number of characters after '.com': {count_characters_after_tld(url)}")
        extracted_features += 1
        print(f"URL Entropy: {calculate_entropy(url):.2f}")
        extracted_features += 1
        print(f"Contains IP Address: {has_ip_address(url)}")
        extracted_features += 1
        print(f"Number of Dots: {count_dots_in_url(url)}")
        extracted_features += 1
        print(f"Number of Hyphens: {count_hyphens_in_domain(url)}")
        extracted_features += 1
        print(f"Special Characters: {count_special_characters(url)}")
        extracted_features += 1
        print(f"Google Safe Browsing: {check_url_safety(url)}")
        extracted_features += 1
        print(f"Number of redirects: {check_url_redirection(url)}")
        extracted_features += 1
        print(f"Suspicious IP in URL: {check_suspicious_ip(url)}")
        extracted_features += 1
        print(f"HTTPS token in URL: {check_https_token_in_url(url)}")
        extracted_features += 1
    except Exception as e:
        print(f"Error in URL Analysis: {e}")

    # WHOIS Analysis
    print("\n--- WHOIS Analysis ---")
    try:
        domain_age = get_domain_age(url)
        if domain_age is not None:
            print(f"Domain Age: {domain_age:.2f} years")
            extracted_features += 1

        domain_expiration = get_domain_expiration(url)
        if domain_expiration is not None:
            print(f"Domain Expiration Time: {domain_expiration} days")
            extracted_features += 1

        registrar_check = check_registrar(url)
        if registrar_check is not None:
            print(f"Registrar check: {registrar_check}")
            extracted_features += 1

        whois_privacy = check_whois_privacy_protection(url)
        if whois_privacy is not None:
            print(f"WHOIS privacy protection: {whois_privacy}")
            extracted_features += 1

        owner_details = check_owner_details(url)
        if owner_details is not None:
            print(f"Owner details check: {owner_details}")
            extracted_features += 1
    except Exception as e:
        print(f"Error in WHOIS Analysis: {e}")

    # SSL Certificate Analysis
    print("\n--- SSL Certificate Analysis ---")
    try:
        ssl_status = check_ssl_certificate(url)
        if ssl_status:
            print(f"SSL Certificate Valid: {'Yes' if ssl_status else 'No'}")
            extracted_features += 1

        certificate_validity, certificate_active = get_certificate_validity(url)
        if certificate_validity:
            print(f"SSL Certificate Validity Period: {certificate_validity} days")
            extracted_features += 1
    except Exception as e:
        print(f"Error in SSL Analysis: {e}")

    # Webpage Content and Analysis
    print("\n--- Webpage Analysis ---")
    try:
        content = get_page_content(url)
        if content:
            num_issues, issues = check_grammar_and_spelling(content)
            print(f"Number of grammatical or spelling issues: {num_issues}")
            extracted_features += 1

            external_links, internal_links = get_page_links(url)
            print(f"Number of external links: {external_links}")
            extracted_features += 1
            print(f"Number of internal links: {internal_links}")
            extracted_features += 1

            login_forms = detect_login_forms(url)
            print(f"Number of login forms: {login_forms}")
            extracted_features += 1

            input_fields, input_types = detect_input_fields(url)
            print(f"Number of input fields: {input_fields}")
            extracted_features += 1

            js_length = measure_javascript_length(url)
            print(f"Total JavaScript length: {js_length} characters")
            extracted_features += 1

            inline_scripts_length, iframe_count = detect_inline_scripts_and_iframes(url)
            print(f"Total length of inline JavaScript: {inline_scripts_length} characters")
            extracted_features += 1
            print(f"Number of iframe elements: {iframe_count}")
            extracted_features += 1

            title_length, meta_desc_length = extract_title_and_meta_description_length(url)
            print(f"Title Length: {title_length} characters")
            extracted_features += 1
            print(f"Meta Description Length: {meta_desc_length} characters")
            extracted_features += 1
        else:
            print("Failed to retrieve page content.")
    except Exception as e:
        print(f"Error in Webpage Analysis: {e}")

    print("\n--- Summary ---")
    print(f"Extracted Features: {extracted_features}/{total_features} successfully")

if __name__ == "__main__":
    main()


Enter a URL: http://galorbg.github.io/
--- URL Analysis ---
URL Length: 25
Number of characters after '.com': 0
URL Entropy: 3.67
Contains IP Address: False
Number of Dots: 2
Number of Hyphens: 0
Special Characters: 3
Google Safe Browsing: Suspicious tld: False
Number of redirects: 1
Suspicious IP in URL: 0
HTTPS token in URL: 0

--- WHOIS Analysis ---
Error retrieving domain age: unsupported operand type(s) for -: 'datetime.datetime' and 'NoneType'
Error retrieving domain expiration: unsupported operand type(s) for -: 'NoneType' and 'datetime.datetime'
Registrar check: 0
WHOIS privacy protection: 0
Owner details check: 1

--- SSL Certificate Analysis ---
SSL Certificate Valid: Yes
SSL Certificate Validity Period: 364 days

--- Webpage Analysis ---
Number of grammatical or spelling issues: 12
Number of external links: 0
Number of internal links: 2
Number of login forms: 0
Number of input fields: 1
Total JavaScript length: 0 characters
Total length of inline JavaScript: 0 characters
Num

In [ ]:
!pip install pandas
!pip install tqdm
!pip install requests
!pip install python-whois
!pip install beautifulsoup4
!pip install language-tool-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 2.0 MB/s eta 0:00:00


In [ ]:
!pip install whois


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 1.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from tqdm import tqdm
import time  # Simulates processing time
import os
from collections import Counter
import math
import re
from urllib.parse import urlparse, urljoin
import requests
import whois
from datetime import datetime
import ssl
import socket
from bs4 import BeautifulSoup
import language_tool_python
from tqdm import tqdm

# Replace with your Google API Key
API_KEY = ''
SAFE_BROWSING_URL = "https://safebrowsing.googleapis.com/v4/threatMatches:find"

# Function Definitions (Merged)

def count_characters_after_tld(url, tld=".com"):
    try:
        if tld in url:
            index = url.find(tld) + len(tld)  # Find the position right after the TLD
            return len(url) - index  # Calculate characters after the TLD
        return 0
    except Exception as e:
        print(f"Error counting characters after TLD: {e}")
        return 0

def calculate_entropy(url):
    frequency = Counter(url)
    total_characters = len(url)
    return -sum((count / total_characters) * math.log2(count / total_characters) for count in frequency.values())

def has_ip_address(url):
    ip_pattern = re.compile(r'\b(?:\d{1,3}\.){3}\d{1,3}\b')
    return bool(ip_pattern.search(url))

def count_dots_in_url(url):
    return url.count('.')

def count_hyphens_in_domain(url):
    domain = urlparse(url).netloc
    return domain.count('-')

def count_special_characters(url):
    special_characters = r'[@%$&!*^+=?/<>|~]'
    matches = re.findall(special_characters, url)
    return len(matches)

def check_url_safety(url):
    payload = {
        "client": {"clientId": "yourClientID", "clientVersion": "1.0"},
        "threatInfo": {
            "threatTypes": ["MALWARE", "SOCIAL_ENGINEERING"],
            "platformTypes": ["ANY_PLATFORM"],
            "threatEntryTypes": ["URL"],
            "threatEntries": [{"url": url}]
        }
    }
    try:
        response = requests.post(f'{SAFE_BROWSING_URL}?key={API_KEY}', json=payload)
        result = response.json()
        return "Suspicious tld: True" if 'matches' in result else "Suspicious tld: False"
    except requests.exceptions.RequestException as e:
        print(f"Error checking URL safety: {e}")
        return "Unknown"

def check_url_redirection(url):
    try:
        response = requests.get(url, allow_redirects=True)
        return len(response.history)
    except requests.exceptions.RequestException as e:
        print(f"Error checking URL redirection: {e}")
        return 0

def check_suspicious_ip(url):
    parsed_url = urlparse(url)
    ipv4_pattern = re.compile(r'^(?:\d{1,3}\.){3}\d{1,3}$')
    ipv6_pattern = re.compile(r'^[0-9a-fA-F:]{2,39}$')
    host = parsed_url.hostname
    if ipv4_pattern.match(host) or ipv6_pattern.match(host):
        return 1
    else:
        return 0

def check_https_token_in_url(url):
    parsed_url = urlparse(url)
    if 'https' in parsed_url.scheme or 'https' in parsed_url.path or 'https' in parsed_url.query:
        return 1
    else:
        return 0

def get_domain_age(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        creation_date = w.creation_date
        if isinstance(creation_date, list):
            creation_date = creation_date[0]
        current_date = datetime.now()
        return (current_date - creation_date).days / 365
    except Exception as e:
        print(f"Error retrieving domain age: {e}")
        return None

def get_domain_expiration(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        expiration_date = w.expiration_date
        if isinstance(expiration_date, list):
            expiration_date = expiration_date[0]
        current_date = datetime.now()
        return (expiration_date - current_date).days
    except Exception as e:
        print(f"Error retrieving domain expiration: {e}")
        return None

def check_registrar(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        registrar = w.registrar
        suspicious_registrars = ["Namecheap Inc.", "GoDaddy.com, LLC", "PublicDomainRegistry.com"]
        if registrar and any(suspicious in registrar for suspicious in suspicious_registrars):
            return 1
        return 0
    except Exception as e:
        print(f"Error retrieving registrar: {e}")
        return None

def check_whois_privacy_protection(url):
    try:
        domain = url.split('/')[2]
        w = whois.whois(domain)
        registrant_info = w.get('registrant', '') + ' ' + w.get('contact', '')
        privacy_protection_indicators = ["Privacy Protected", "Domains by Proxy"]
        if any(indicator in registrant_info for indicator in privacy_protection_indicators):
            return 1
        return 0
    except Exception as e:
        print(f"Error checking WHOIS privacy: {e}")
        return None

def check_owner_details(url):
    try:
        domain = url.split('/')[2]
        if domain.endswith('google.com'):
            return 0
        w = whois.whois(domain)
        if not w.get('registrant_name') or not w.get('registrant_email'):
            return 1
        return 0
    except Exception as e:
        print(f"Error checking owner details: {e}")
        return None

def check_ssl_certificate(url):
    try:
        domain = url.split('/')[2]
        conn = ssl.create_default_context().wrap_socket(socket.socket(), server_hostname=domain)
        conn.connect((domain, 443))
        cert = conn.getpeercert()
        not_after = datetime.strptime(cert['notAfter'], "%b %d %H:%M:%S %Y GMT")
        return not_after > datetime.now()
    except Exception as e:
        print(f"Error checking SSL certificate: {e}")
        return False

def get_certificate_validity(url):
    try:
        domain = url.split('/')[2]
        conn = ssl.create_default_context().wrap_socket(socket.socket(), server_hostname=domain)
        conn.connect((domain, 443))
        cert = conn.getpeercert()
        not_before = datetime.strptime(cert['notBefore'], "%b %d %H:%M:%S %Y GMT")
        not_after = datetime.strptime(cert['notAfter'], "%b %d %H:%M:%S %Y GMT")
        validity_duration = (not_after - not_before).days
        return validity_duration, not_after > datetime.now()
    except Exception as e:
        print(f"Error checking certificate validity: {e}")
        return None, False

def get_page_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup.get_text()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return ""

def check_grammar_and_spelling(text):
    tool = language_tool_python.LanguageTool('en-US')
    matches = tool.check(text)
    return len(matches), matches

def get_page_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a', href=True)
        base_domain = urlparse(url).netloc
        external_links = sum(1 for link in links if urlparse(urljoin(url, link['href'])).netloc != base_domain)
        internal_links = sum(1 for link in links if urlparse(urljoin(url, link['href'])).netloc == base_domain)
        return external_links, internal_links
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def detect_login_forms(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        forms = soup.find_all('form')
        return sum(1 for form in forms if form.find_all('input', {'name': lambda x: x and ('user' in x or 'password' in x)}))
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0

def detect_input_fields(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        input_fields = soup.find_all('input')
        input_types = {}
        for input_field in input_fields:
            input_type = input_field.get('type', 'text')
            input_types[input_type] = input_types.get(input_type, 0) + 1
        return len(input_fields), input_types
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, {}

def measure_javascript_length(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        script_tags = soup.find_all('script')
        return sum(len(script.string) for script in script_tags if script.string)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0

def detect_inline_scripts_and_iframes(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        inline_scripts_length = sum(len(script.string) for script in soup.find_all('script') if script.string)
        iframe_count = len(soup.find_all('iframe'))
        return inline_scripts_length, iframe_count
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0

def extract_title_and_meta_description_length(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        title_tag = soup.find('title')
        meta_description_tag = soup.find('meta', attrs={'name': 'description'})
        return len(title_tag.string) if title_tag else 0, len(meta_description_tag['content']) if meta_description_tag else 0
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return 0, 0
def extract_features(url):
    try:
        return {
            # URL Analysis Features
            "URL Length": len(url),
            "Characters After .com": count_characters_after_tld(url),
            "Entropy": calculate_entropy(url),
            "Contains IP": has_ip_address(url),
            "Number of Dots": count_dots_in_url(url),
            "Number of Hyphens": count_hyphens_in_domain(url),
            "Special Characters": count_special_characters(url),
            "Redirections": check_url_redirection(url),
            "Suspicious IP in URL": check_suspicious_ip(url),
            "HTTPS Token in URL": check_https_token_in_url(url),

            # WHOIS Analysis Features
            "Domain Age": get_domain_age(url),
            "Domain Expiration": get_domain_expiration(url),
            "Registrar Check": check_registrar(url),
            "WHOIS Privacy": check_whois_privacy_protection(url),
            "Owner Details": check_owner_details(url),

            # SSL Certificate Analysis Features
            "SSL Certificate Valid": check_ssl_certificate(url),
            "SSL Certificate Validity": get_certificate_validity(url),

            # Webpage Content Analysis Features
            "Number of Grammar Issues": check_grammar_and_spelling(get_page_content(url))[0],
            "Number of External Links": get_page_links(url)[0],
            "Number of Internal Links": get_page_links(url)[1],
            "Number of Login Forms": detect_login_forms(url),
            "Number of Input Fields": detect_input_fields(url)[0],
            "JavaScript Length": measure_javascript_length(url),
            "Inline JS Length": detect_inline_scripts_and_iframes(url)[0],
            "Number of Iframes": detect_inline_scripts_and_iframes(url)[1],
            "Title Length": extract_title_and_meta_description_length(url)[0],
            "Meta Description Length": extract_title_and_meta_description_length(url)[1],
        }
    except Exception as e:
        print(f"Error extracting features for URL {url}: {e}")
        return {}

def process_csv(input_file, output_file):
    if not os.path.exists(input_file):
        print(f"Error: Input file {input_file} not found.")
        return

    try:
        # Read the input CSV
        df = pd.read_csv(input_file)
        if 'URL' not in df.columns or 'Label' not in df.columns:
            print("Error: Input file must contain 'URL' and 'Label' columns.")
            return

        processed_data = []

        # Process each URL
        for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing URLs"):
            url = row['URL']
            label = row['Label']

            # Extract features
            features = extract_features(url)
            features.update({"URL": url, "Label": label})

            # Append to processed data
            processed_data.append(features)

        # Convert to DataFrame
        processed_df = pd.DataFrame(processed_data)

        # Ensure all 27 features are present
        expected_features = [
            "URL Length", "Characters After .com", "Entropy", "Contains IP", "Number of Dots", "Number of Hyphens",
            "Special Characters", "Redirections", "Suspicious IP in URL", "HTTPS Token in URL", "Domain Age",
            "Domain Expiration", "Registrar Check", "WHOIS Privacy", "Owner Details", "SSL Certificate Valid",
            "SSL Certificate Validity", "Number of Grammar Issues", "Number of External Links", "Number of Internal Links",
            "Number of Login Forms", "Number of Input Fields", "JavaScript Length", "Inline JS Length", "Number of Iframes",
            "Title Length", "Meta Description Length"
        ]

        for feature in expected_features:
            if feature not in processed_df.columns:
                processed_df[feature] = None  # Add missing feature as None

        # Save to CSV
        processed_df.to_csv(output_file, index=False)
        print(f"Processed data saved to {output_file}.")

    except Exception as e:
        print(f"Error processing CSV: {e}")

if __name__ == "__main__":
    input_csv = "active_urls_async_test.csv"  # Replace with your input file
    output_csv = "Phishy.csv"

    process_csv(input_csv, output_csv)

Error: Input file must contain 'URL' and 'Label' columns.


In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 28.9 MB/s eta 0:00:00


In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

# Initialize Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)

# Base URL
base_url = "https://phishtank.org/phish_archive.php"

# Store extracted URLs
valid_phish_urls = []

# Open the website
driver.get(base_url)

while len(valid_phish_urls) < 5000:
    try:
        # Wait for the table to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "phish_archive"))
        )

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Find all rows in the table
        rows = soup.select("tr")  # All rows in the table

        for row in rows:
            columns = row.find_all("td")
            if len(columns) >= 4:
                valid_status = columns[2].text.strip()
                online_status = columns[3].text.strip()
                if valid_status == "VALID PHISH" and online_status == "ONLINE":
                    phish_url = columns[1].find("a").text.strip()
                    valid_phish_urls.append(phish_url)

                    # Stop if we've reached 5000 URLs
                    if len(valid_phish_urls) >= 5000:
                        break

        # If there's an "Older" button, click it
        next_button = driver.find_element(By.LINK_TEXT, "Older")
        if next_button:
            next_button.click()
            time.sleep(3)  # Give some time for the page to load
        else:
            break  # No more pages to scrape

    except Exception as e:
        print(f"Error: {e}")
        break

# Close the driver
driver.quit()

# Save the URLs to a CSV file
df = pd.DataFrame(valid_phish_urls, columns=["Phish URL"])
df.to_csv("valid_phish_urls.csv", index=False)

print(f"Scraping completed. {len(valid_phish_urls)} URLs saved to valid_phish_urls.csv.")


Error: Message: 
Stacktrace:
#0 0x55fe1e3308fa <unknown>
#1 0x55fe1de41d20 <unknown>
#2 0x55fe1de90a66 <unknown>
#3 0x55fe1de90d01 <unknown>
#4 0x55fe1ded6184 <unknown>
#5 0x55fe1deb4b1d <unknown>
#6 0x55fe1ded3560 <unknown>
#7 0x55fe1deb4893 <unknown>
#8 0x55fe1de8330d <unknown>
#9 0x55fe1de8432e <unknown>
#10 0x55fe1e2fd00b <unknown>
#11 0x55fe1e300f97 <unknown>
#12 0x55fe1e2e971c <unknown>
#13 0x55fe1e301b17 <unknown>
#14 0x55fe1e2ce6cf <unknown>
#15 0x55fe1e31f6b8 <unknown>
#16 0x55fe1e31f880 <unknown>
#17 0x55fe1e32f776 <unknown>
#18 0x78f4d46a7ac3 <unknown>

Scraping completed. 0 URLs saved to valid_phish_urls.csv.


In [ ]:
import csv
import requests
from requests.exceptions import RequestException

def is_url_active(url):
    """Check if a URL is active by making an HTTP GET request."""
    try:
        response = requests.head(url, timeout=5)  # Use HEAD request for faster response
        return response.status_code == 200
    except RequestException:
        return False

def process_urls(input_csv, output_csv):
    """Process URLs to check if active and save active URLs with their labels."""
    active_urls = []

    # Read the input CSV file
    with open(input_csv, 'r', newline='', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)

        # Check if required columns exist
        if 'url' not in reader.fieldnames or 'label' not in reader.fieldnames:
            raise ValueError("Input CSV must contain 'url' and 'label' columns.")

        for row in reader:
            url = row['url']
            label = row['label']

            if is_url_active(url):
                active_urls.append({'url': url, 'label': label})

    # Write active URLs to the output CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as outfile:
        fieldnames = ['url', 'label']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(active_urls)

if __name__ == "__main__":
    input_csv = "input.csv"  # Replace with your input CSV file path
    output_csv = "active_urls.csv"  # Replace with your desired output CSV file path

    try:
        process_urls(input_csv, output_csv)
        print(f"Active URLs saved to {output_csv}")
    except Exception as e:
        print(f"Error: {e}")


Error: name 'status' is not defined


In [ ]:
pip install aiohttp tqdm nest_asyncio


In [ ]:
import pandas as pd
from tqdm import tqdm
import os
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Function to extract features (you can include additional feature extraction logic here)
def extract_features(url):
    # Example features; customize as needed
    return {
        "url_length": len(url),
        "contains_https": "https" in url,
    }

# Function to process the CSV
def process_csv(input_file, output_file):
    if not os.path.exists(input_file):
        print(f"Error: Input file {input_file} not found.")
        return

    try:
        # Read the input CSV
        df = pd.read_csv(input_file)
        if 'url' not in df.columns or 'parent_status' not in df.columns:
            print("Error: Input file must contain 'url' and 'parent_status' columns.")
            return

        processed_data = []

        # Create a requests session with retries
        session = requests.Session()
        retries = Retry(total=5, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
        session.mount('http://', HTTPAdapter(max_retries=retries))
        session.mount('https://', HTTPAdapter(max_retries=retries))

        # Process each URL
        for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing URLs"):
            url = row['url']
            parent_status = row['parent_status']

            try:
                # Send a GET request to the URL
                response = session.get(url, timeout=15)
                if response.status_code == 200:
                    features = extract_features(url)
                    features.update({"url": url, "parent_status": parent_status})
                    processed_data.append(features)
                else:
                    print(f"URL {url} returned status code {response.status_code}")
            except Exception as e:
                print(f"Error with URL {url}: {e}")
                continue

        # Convert processed data to DataFrame
        processed_df = pd.DataFrame(processed_data)

        # Save the processed data to a CSV file
        processed_df.to_csv(output_file, index=False)
        print(f"Processed data saved to {output_file}.")

    except Exception as e:
        print(f"Error processing CSV: {e}")

# Main script
if __name__ == "__main__":
    input_csv = "final_cleaned_with_status.csv"  # Replace with your input file
    output_csv = "Phishy.csv"                # Replace with your desired output file

    process_csv(input_csv, output_csv)


Processing URLs:   0%|          | 1/6006 [00:33<55:26:59, 33.24s/it]

Error with URL http://www.crestonwood.com/router.php: HTTPConnectionPool(host='www.crestonwood.com', port=80): Max retries exceeded with url: /router.php (Caused by ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))


Processing URLs:   0%|          | 9/6006 [02:12<29:02:42, 17.44s/it]

Error with URL http://wave.progressfilm.co.uk/time3/?logon=myposte: HTTPConnectionPool(host='wave.progressfilm.co.uk', port=80): Max retries exceeded with url: /time3/?logon=myposte (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7812fa4e0670>: Failed to resolve 'wave.progressfilm.co.uk' ([Errno -2] Name or service not known)"))


Processing URLs:   1%|          | 34/6006 [02:57<11:57:17,  7.21s/it]

...

Processing URLs:  66%|██████▌   | 3938/6006 [4:07:00<6:26:28, 11.21s/it]

Error with URL https://www.eayso.org/: HTTPSConnectionPool(host='www.eayso.org', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate (_ssl.c:1007)')))


Processing URLs:  66%|██████▌   | 3940/6006 [4:07:31<8:28:35, 14.77s/it]

Error with URL http://cmffunding.com/wp-admin/includes/AT&T/Att.Yahoo/confirmation.html: HTTPConnectionPool(host='cmffunding.com', port=80): Max retries exceeded with url: /wp-admin/includes/AT&T/Att.Yahoo/confirmation.html (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7812f933e650>: Failed to resolve 'cmffunding.com' ([Errno -2] Name or service not known)"))


Processing URLs:  66%|██████▌   | 3946/6006 [4:07:35<1:19:03,  2.30s/it]